In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
fname = "relevant_json.json"
list_tweets = []
count = 0 
sum = 1000
with open(fname,encoding="utf8") as f:
    for line in f:
        try:
            tweet = json.loads(line)
        except:
            pass
        list_tweets.append(tweet)


df_relevant = pd.DataFrame(data=[tweet["full_text"] for tweet in list_tweets], columns=['tweets'])
df_relevant['created_at'] = np.array([tweet["created_at"] for tweet in list_tweets])
df_relevant['id'] = np.array([int(tweet["id_str"]) for tweet in list_tweets])
df_relevant['rt'] = np.array([tweet["retweeted"] for tweet in list_tweets])
df_relevant['lang'] = np.array([tweet["lang"] for tweet in list_tweets])
df_relevant['Location'] = None
df_relevant['coordinates'] = None
# df_relevant['geo'] = np.array([tweet["geo"] for tweet in list_tweets])
# df_relevant['coordinates'] = np.array([tweet["coordinates"] for tweet in list_tweets])

# df_relevant.shape[0]

fname = "relevant_tweets.tsv"
event_id = pd.read_csv(fname, sep='\t',header=None,names=['Event','id'])

# event_id.shape[0]

df_relevant = df_relevant.merge(event_id, left_on='id', right_on='id')
df_relevant.head(1)

,tweets,created_at,id,rt,lang,Location,coordinates,Event
0,News: Chinese author Mo Yan wins Nobel http://...,Thu Oct 11 11:08:32 +0000 2012,256350567369175040,False,en,None,None,0


In [4]:
import requests
list_cities = ["Manchester","London","Liverpool","Glasgow","Leeds","Southampton","Birmingham","Bristol","Lancaster"
              ,"Leicester","Wolverhampton","Sheffield","Dublin","Aberdeen","Delft","Amsterdam","Groningen","Eindhoven",
               "Leuven","Ghent","Brussels","Dortmund","Cologne","Odense","Copenhagen","Stockholm","Oslo","Helsinki",
               "Bucharest","Sofia","Milan","Rome","Athens","Barcelona","Madrid","Seville","Lisbon","Porto","Prague","Warsaw",
               "Riga","Edinburgh","New York","Chicago","Atlanta","Detroit","Dallas"
              ]
list_coordinates = []
for city in list_cities:
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search?city=',city,'&format=json&polygon=0')
    response = requests.get(url).json()[0]
    bounding_box = response['boundingbox']
    latmin = float(bounding_box[0])
    latmax = float(bounding_box[1])
    lonmin = float(bounding_box[2])
    lonmax = float(bounding_box[3])
    x_object = {"city":city,"latmin":latmin,"latmax":latmax,"lonmin":lonmin,"lonmax":lonmax}
    list_coordinates.append(x_object)
# list_coordinates = [[-71.513203,42.890351,-71.375431,43.051483],[-0.5104,51.2868,0.334,51.6919],
#                    [150.898112,-33.939026,150.934437,-33.903906],[-4.393201,55.781277,-4.071717,55.929639],
#                    [-1.800421,53.698967,-1.290352,53.945871],[-1.479005,50.871509,-1.321986,50.956131],
#                    [-2.033649,52.381053,-1.728858,52.608706],[-2.71837,51.397284,-2.510419,51.544432],
#                    [-76.346548,40.00691,-76.254092,40.073034],[-71.9606,42.189313,-71.856792,42.28454],
#                    [-2.20688172,52.54391315,-2.04821213,52.63789466],[-1.80147161,53.30451152,-1.32466856,53.50312843],
#                    [-121.98779772,37.69775717,-121.83243906,37.74509958],[-123.86050365,46.94361772,-123.68072708,47.02059],
#                    [4.32021776,51.96631629,4.40791082,52.03259879],[4.72924173,52.27817413,5.07916213,52.43106373],
#                    [6.46274553,53.17861022,6.66687262,53.26494684],[5.35671837,51.40004748,5.54885272,51.49707782],
#                    [4.64029498,50.82420956,4.77053048,50.94407069],[3.57976152,50.9796393,3.84932495,51.18794594],
#                    [4.33549889,50.79640788,4.40201147,50.89041076],[7.30244216,51.41552542,7.638157,51.60004145],
#                    [6.77253028,50.83044939,7.16202793,51.08497426],[10.1758237,55.28811062,10.57682928,55.48198898],
#                    [-75.68698501,43.88192894,-75.65920151,43.90525594],[17.76069212,59.22761885,18.20001876,59.44007814],
#                    [10.48916514,59.80931128,10.95138938,60.13510639],[24.78281305,59.92248592,25.25451153,60.29784967],
#                    [25.96474455,44.33619557,26.22742099,44.54196014],[23.19468464,42.62007935,23.42635717,42.78678593],
#                    [9.04088668,45.38673805,9.27811022,45.53584817],[-85.27894104,34.19315898,-85.11224603,34.35649998],
#                    [2.05249763,41.31703522,2.22835549,41.46825196],[-3.88895392,40.31197738,-3.51791632,40.64372926],
#                    [-6.03291835,37.30020359,-5.81915711,37.45295782],[-9.22983565,38.69139935,-9.08633286,38.79675837],
#                    [-8.69129418,41.13835064,-8.55200908,41.18593529],[14.22443548,49.94190059,14.70678685,50.17743006],
#                    [-76.79536304,37.943847,-76.73735306,37.97443692],[23.93237049,56.85734614,24.32484355,57.08598149],
#                    [-3.4055998972,55.8638336236,-3.1137755563,55.9750313038],[-74.25909008,40.47739894,-73.70018092,40.91617849],
#                    [-87.94010101,41.64391896,-87.5239841,42.02302188],[-84.55106806,33.64780797,-84.28956005,33.88682297],
#                    [-83.287959,42.25519197,-82.91043917,42.45023198],[-97.00048203,32.613216,-96.46363177,33.02393655],
#                    [23.4944803797,37.8996636641,23.8899881922,38.1808721747],[-124.48200307,32.52952353,-114.13078164,42.00949894]]

In [5]:
len(list_coordinates)

47

In [6]:
list_labels = df_relevant.Event.unique()

In [7]:
import sys
import random
list_df = []
for current_label in list_labels:
    dfchange = df_relevant.loc[df_relevant["Event"]==current_label]
    location = random.choice(list_coordinates)
    dfchange["Location"] = location['city']
    list_df.append(dfchange)

C:\Users\giann\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [8]:
list_df[1]

,tweets,created_at,id,rt,lang,Location,coordinates,Event
252,“@JoeBiden: RT if you’re on #TeamObamaBiden to...,Thu Oct 11 21:20:13 +0000 2012,256504502662873088,False,en,Edinburgh,None,1
333,Make sure to watch the vice presidential debat...,Thu Oct 11 23:27:57 +0000 2012,256536647615803392,False,en,Edinburgh,None,1
334,Make sure to watch the vice presidential debat...,Thu Oct 11 23:27:57 +0000 2012,256536647615803392,False,en,Edinburgh,None,1
335,Paul Ryan and Joe Biden: (Don't) Believe the H...,Thu Oct 11 22:59:21 +0000 2012,256529450542456833,False,en,Edinburgh,None,1
336,Vice Presidential debate tonight....dear lord ...,Thu Oct 11 23:38:46 +0000 2012,256539370000105473,False,en,Edinburgh,None,1
...,...,...,...,...,...,...,...,...
6962,Ryan kept talking. he claims Obama makes spe...,Fri Oct 12 02:51:42 +0000 2012,256587923145777152,False,en,Edinburgh,None,1
6963,Paul Ryan says believes life begins at concept...,Fri Oct 12 02:52:24 +0000 2012,256588099415592961,False,en,Edinburgh,None,1
6964,Paul Ryan says believes life begins at concept...,Fri Oct 12 02:52:24 +0000 2012,256588099415592961,False,en,Edinburgh,None,1
7039,Who won the debate? Actually I don't really c...,Fri Oct 12 02:59:50 +0000 2012,256589970049994753,False,en,Edinburgh,None,1


In [9]:
import requests
def setCoordinates(city,list_coordinates):
    x_object = None
    for temp in list_coordinates:
        if (city in temp['city']):
            random_lat = random.uniform(temp['latmin'],temp['latmax'])
            random_lon = random.uniform(temp['lonmin'],temp['lonmax'])
            x_object = {"lat":random_lat,"lon":random_lon}
            return json.dumps(x_object)
    return json.dumps(x_object)

In [10]:
for dfchange in list_df:
    dfchange["coordinates"] = np.array([setCoordinates(temp,list_coordinates) for temp in dfchange["Location"]])

C:\Users\giann\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
total_df = pd.concat(list_df)

In [13]:
total_df.to_csv(r'relevant_with_coordinates.csv',index=False,header=False)

In [12]:
# count = 100
# for i in range(len(list_df)):
#     city = list_df[i].groupby(["Location"]).sum().index[0]
#     url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search?city=',city,'&format=json&polygon=0')
#     response = requests.get(url).json()[0]
#     bounding_box = response['boundingbox']
#     latmin = float(bounding_box[0])
#     latmax = float(bounding_box[1])
#     lonmin = float(bounding_box[2])
#     lonmax = float(bounding_box[3])
#     for index,row in list_df[i].iterrows():
#         row["coordinates"] = setCoordinates(latmin,latmax,lonmin,lonmax)
#     if (i==count):
#         print(i)
#         count+=100

In [ ]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
# coordinates = "55.83048439111454,-4.249950675"
coordinates = "42.40296141598293, -82.98640647"
location = locator.reverse(coordinates)
location.raw